In [1]:
import os
from os import environ
import jupyterpost
from jupyterpost import post
os.environ["JPY_API_TOKEN"] = "d0797e84f32243aebbb3711203615dab"
os.environ["JUPYTERPOST_URL"] = "https://services.io.quantumtinkerer.group/services/jupyterpost"

In [2]:
from itertools import count

import sympy
import numpy as np
import tinyarray as ta

from polynomial_orders_U import compute_next_orders, H_tilde
from sympy import MatrixSymbol
from sympy.physics.quantum.dagger import Dagger

In [3]:
wanted_orders = [[4]]
Ns = [2, 2] # np.random.randint(1, high=5, size=2)


def hamiltonians(Ns, wanted_orders):
    """
    Produce random Hamiltonians to test.

    Ns: dimension of each block (A, B)
    wanted_orders: list of orders to compute

    Returns:
    hams: list of Hamiltonians
    """
    N_p = len(wanted_orders[0])
    orders = ta.array(np.eye(N_p))
    hams = []
    for i in range(2):
        hams.append(np.diag(np.sort(np.random.rand(Ns[i])) - i))

    def matrices_it(N_i, N_j, hermitian):
        """
        Generate random matrices of size N_i x N_j.

        N_i: number of rows
        N_j: number of columns
        hermitian: if True, the matrix is hermitian

        Returns:
        generator of random matrices
        """
        for i in count():
            H = np.random.rand(N_i, N_j) + 1j * np.random.rand(N_i, N_j)
            if hermitian:
                H += H.conj().T
            yield H

    for i, j, hermitian in zip([0, 1, 0], [0, 1, 1], [True, True, False]):
        matrices = matrices_it(Ns[i], Ns[j], hermitian)
        hams.append({order: matrix for order, matrix in zip(orders, matrices)})
    return hams



In [4]:
hams = hamiltonians(Ns, wanted_orders)
H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB = hams
exp_S = compute_next_orders(*hams, wanted_orders=wanted_orders)

H_AA_01 = H_tilde(H_0_AA, np.zeros_like(H_0_BB), {}, {}, {}, wanted_orders, exp_S)[0]
H_AA_02 = H_tilde(H_0_AA, H_0_BB, {}, {}, {}, wanted_orders, exp_S)[0]
H_AA_p1 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), H_p_AA, {}, {}, wanted_orders, exp_S)[0]
H_AA_p2 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), {}, H_p_BB, {}, wanted_orders, exp_S)[0]
H_AA_p3 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), {}, {}, H_p_AB, wanted_orders, exp_S)[0]

In [5]:
# test
from sympy.physics.quantum import Operator, HermitianOperator
from operator import mul

dim1 = 2
dim2 = 2

H_0_AA = HermitianOperator("{H_{0}^{AA}}")
H_0_BB = HermitianOperator("{H_{0}^{BB}}")

H_p_AA = {}
H_p_BB = {}
H_p_AB = {}

H_p_AA[ta.array([1])] = HermitianOperator("{H_{p}^{AA}}")
H_p_BB[ta.array([1])] = HermitianOperator("{H_{p}^{BB}}")   
H_p_AB[ta.array([1])] = Operator("{H_{p}^{AB}}")

In [6]:
class EnergyDivider:
    def __init__(self):
        self.data = {}

    def __call__(self, rhs):
        new_entry = Operator(f"V_{len(self.data) + 1}")
        self.data[new_entry] = rhs
        return new_entry

In [7]:
wanted_orders=[[5]]
divider = EnergyDivider()
exp_S = compute_next_orders(
    H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB,
    wanted_orders=wanted_orders,
    divide_energies=divider,
    op=mul,
)


In [8]:
H_tilde_AA, H_tilde_BB = H_tilde(
    H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB, wanted_orders, exp_S, op=mul
)


In [9]:
# sympy.simplify(list(divider.data.values())[-1])

In [10]:
def simplify_recursively(H_0_AA, H_0_BB, dictionary, divider_data):
    i = 1
    for key in dictionary.keys():
        for _ in range(i):
            value = dictionary[key]
            value = sympy.expand(value.subs({H_0_AA * v: v * H_0_BB + rhs for v, rhs in divider_data.items()})) # H_0_AA to right
            value = sympy.expand(value.subs({H_0_BB * Dagger(v): Dagger(v) * H_0_AA - Dagger(rhs) for v, rhs in divider_data.items()})) # H_0_BB to right
            dictionary.update({key: value})
        # display(dictionary[key])
        i += 1
    return dictionary

In [17]:
# list(divider.data.values())[2]

In [12]:
divider.data = simplify_recursively(H_0_AA, H_0_BB, divider.data, divider.data)

In [18]:
# list(divider.data.values())[2]

In [19]:
# list(H_tilde_AA.values())[2]

In [14]:
H_tilde_AA = simplify_recursively(H_0_AA, H_0_BB, H_tilde_AA, divider.data)

In [20]:
# list(H_tilde_AA.values())[2]